In [1]:
\l ml/ml.q
.ml.loadfile`:init.q
.ml.loadfile`:clust/init.q

In [2]:
\l ../automl/automl.q
.aml.loadfile`:init.q

In [3]:
avs:.p.import[`sklearn.metrics]`:average_precision_score
svc:.p.import[`sklearn.svm]`:SVC
array:.p.import[`numpy]`:array
mattab:{flip value flip x}

* Why the choice to standardize to 7 characters to 8 and ignore the rest, there are some sites with 9 ... 15 etc
* Some duplicate sites exist with the same latitude and longitude but different site number

In [4]:
ldfn:{("S",(5#"F"),"SFFSFSFFFFSFSSFSSFFFSFFSFF";enlist ",")0:`$":data/snap_sampled_imp_nlcd_20",x,".csv"}
conv7_fn:{`$$[7=count x;"0",;]x}

In [5]:
basin:("S",242#"F";enlist ",")0:`:data/gages_with_basin_attr.csv
gauges:("SSSSFFSSIFFFFFFFFSSISSSSFF";enlist ",")0:`:data/usgs_gage_subset.csv
nlcd06:ldfn"06";nlcd11:ldfn"11";nlcd16:ldfn"16" 
warnings:.ml.df2tab .p.import[`geopandas;`:read_file]"data/national_shapefile_obs.shp"

In [6]:
\l gagesdir/gagesdir

In [7]:
maxht:0!select max height by`$site_no,date from str
maxht:0!select by site_no,"m"$date from delete from maxht where height<0
gauges:update conv7_fn each string each site_no from gauges
show httab:maxht ij 1!select site_no,dec_lat_va,dec_long_v from gauges

site_no  date    height dec_lat_va dec_long_v
---------------------------------------------
01200000 2009.07 3.8    41.65876   -73.52868 
01200000 2009.08 1.82   41.65876   -73.52868 
01200000 2009.09 1.29   41.65876   -73.52868 
01200000 2009.10 2.04   41.65876   -73.52868 
01200000 2009.11 1.71   41.65876   -73.52868 
01200000 2009.12 4.17   41.65876   -73.52868 
01200000 2010.01 2.8    41.65876   -73.52868 
01200000 2010.02 4.35   41.65876   -73.52868 
01200000 2010.03 6.17   41.65876   -73.52868 
01200000 2010.04 1.96   41.65876   -73.52868 
01200000 2010.05 1.22   41.65876   -73.52868 
01200000 2010.06 0.91   41.65876   -73.52868 
01200000 2010.07 0.76   41.65876   -73.52868 
01200000 2010.08 0.83   41.65876   -73.52868 
01200000 2010.09 1.5    41.65876   -73.52868 
01200000 2010.10 1.63   41.65876   -73.52868 
01200000 2010.11 1.8    41.65876   -73.52868 
01200000 2010.12 2.05   41.65876   -73.52868 
01200000 2011.01 2.15   41.65876   -73.52868 
01200000 2011.02 3.75   41.65876  

In [8]:
fp:hsym`$dir,/:string key`$dir:":../../data/prism/"
precipall:raze{`site_no`long`lat`elv`date`ppt xcol 10_("SFFFDF";enlist ",")0:x}each fp
show precipall:update conv7_fn each string each site_no from precipall

site_no  long     lat     elv  date       ppt 
----------------------------------------------
01367690 -74.5596 41.1053 1056 2019.01.01 0.94
01367690 -74.5596 41.1053 1056 2019.01.02 0   
01367690 -74.5596 41.1053 1056 2019.01.03 0   
01367690 -74.5596 41.1053 1056 2019.01.04 0   
01367690 -74.5596 41.1053 1056 2019.01.05 0.38
01367690 -74.5596 41.1053 1056 2019.01.06 0.79
01367690 -74.5596 41.1053 1056 2019.01.07 0   
01367690 -74.5596 41.1053 1056 2019.01.08 0.02
01367690 -74.5596 41.1053 1056 2019.01.09 0.15
01367690 -74.5596 41.1053 1056 2019.01.10 0   
01367690 -74.5596 41.1053 1056 2019.01.11 0   
01367690 -74.5596 41.1053 1056 2019.01.12 0   
01367690 -74.5596 41.1053 1056 2019.01.13 0   
01367690 -74.5596 41.1053 1056 2019.01.14 0   
01367690 -74.5596 41.1053 1056 2019.01.15 0   
01367690 -74.5596 41.1053 1056 2019.01.16 0   
01367690 -74.5596 41.1053 1056 2019.01.17 0   
01367690 -74.5596 41.1053 1056 2019.01.18 0.06
01367690 -74.5596 41.1053 1056 2019.01.19 0   
01367690 -74.

In [9]:
ds:distinct gauges`site_no
rms:asc ds where 12<count each string each ds
names:0!select i by site_no from precipall where site_no in `$12#'string each rms
matchnames:0!select i by lat,long from precipall where i in names[`x][20]

In [10]:
lst1:til[26]except 21 22
lst2:til[25]except 20
i1:rms[lst1],rms[21 22]
i2:(names[`x]lst2),matchnames`x
precipall:{![x;enlist(in;`i;z);0b;enlist[`site_no]!enlist enlist y]}/[precipall;i1;i2]

In [11]:
precipall:update"m"$date from precipall

In [12]:
fn2use:update valid:0b from .ml.fresh.params where pnum>0
cfeats:.ml.fresh.createfeatures[precipall;`site_no`date;`ppt;fn2use]

/home/deanna/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/deanna/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


In [13]:
rainmonth:1_0!cfeats,'select first distinct lat,first distinct long,elevation:distinct elv by site_no,date from precipall

In [14]:
ansf:{where 0<>count each ss[;y]each string each cols x}
constcols:cb til[count cb:cols basin]except raze ansf[basin]each("2009";"2010";"2011")
basinupd:flip constcols!basin constcols
show joinedtab:rainmonth ij`site_no xkey basinupd

site_no  date    ppt_absenergy ppt_abssumchange ppt_count ppt_countabovemean ..
-----------------------------------------------------------------------------..
01200000 2009.07 8.3659        11.36            31        10                 ..
01200000 2009.08 5.0394        7.11             31        9                  ..
01200000 2009.09 0.8817        2.46             30        4                  ..
01200000 2009.10 3.7711        8                31        8                  ..
01200000 2009.11 0.4824        1.94             30        6                  ..
01200000 2009.12 3.7491        6.75             31        9                  ..
01200000 2010.01 3.2776        5.01             31        5                  ..
01200000 2010.02 6.1295        5.87             28        5                  ..
01200000 2010.03 6.6804        6.84             31        6                  ..
01200000 2010.04 0.7151        2.46             30        8                  ..
01200000 2010.05 0.6951        2.84     

In [15]:
col_list:`site_no`INTPTLAT`INTPTLON`Measure`REACHCODE`distance`imp
sym_list:`SOURCE_FEA`INTPTLAT`INTPTLON`Measure`REACHCODE`distance
imp_list:`imp_nlcd_2006`imp_nlcd_2011`imp_nlcd_2016
tab_list:(nlcd06;nlcd11;nlcd16)
nl_tab:raze{[x;y;z;r]?[r;();0b;x!y,z]}[col_list;sym_list]'[imp_list;tab_list]
nl_tab:1!update site_no:conv7_fn each string each site_no from nl_tab

In [16]:
show newjoinedtab:(update year:{$[x<2011;6;x<2016;11;16]}each`year$date from joinedtab)ij nl_tab

site_no  date    ppt_absenergy ppt_abssumchange ppt_count ppt_countabovemean ..
-----------------------------------------------------------------------------..
01200000 2009.07 8.3659        11.36            31        10                 ..
01200000 2009.08 5.0394        7.11             31        9                  ..
01200000 2009.09 0.8817        2.46             30        4                  ..
01200000 2009.10 3.7711        8                31        8                  ..
01200000 2009.11 0.4824        1.94             30        6                  ..
01200000 2009.12 3.7491        6.75             31        9                  ..
01200000 2010.01 3.2776        5.01             31        5                  ..
01200000 2010.02 6.1295        5.87             28        5                  ..
01200000 2010.03 6.6804        6.84             31        6                  ..
01200000 2010.04 0.7151        2.46             30        8                  ..
01200000 2010.05 0.6951        2.84     

In [17]:
del_ind:{first z _asc?[x;();y;`i]}[`warnings]
delac:del_ind[`Action;0]
delmj:del_ind[`Major;1]
delmd:del_ind[`Moderate;2]
delfl:del_ind[`Flood;2]

In [18]:
del_idx:distinct delac,delmj,delmd,delfl
wrn:select from warnings where not i in del_idx
warning:update nn:i from wrn

In [19]:
wlatl:gauges[`dec_lat_va`dec_long_v],'`float$'warning`Latitude`Longitude
tabw:.ml.clust.kd.buildtree[wlatl;2]
gauge_val:count[warning]+til count gauges
nnwarn:.ml.clust.kd.i.nns[;tabw;(count[warning]#0),count[gauges]#1;flip wlatl;`edist]each gauge_val

In [20]:
joins:([site_no:gauges`site_no]nn:nnwarn[;0];ndw:nnwarn[;1])
floodlvl:(maxht ij joins)lj`nn xkey warning
floodlvl[`Action`Moderate`Flood`Major]:"F"$'floodlvl[`Action`Moderate`Flood`Major]
floodlvl:update conv7_fn each string each site_no from floodlvl

In [21]:
show 0!target:select distinct Action,distinct Flood,distinct Moderate,distinct Major,
       no_Action:count where height>Action,no_Flood:count where height>Flood,
       no_Mod:count where height>Moderate,no_Major:count where height>Major 
       by site_no,"m"$date 
       from floodlvl

site_no  date    Action Flood Moderate Major no_Action no_Flood no_Mod no_Major
-------------------------------------------------------------------------------
01200000 2009.07 17     19    22       24    0         0        0      0       
01200000 2009.08 17     19    22       24    0         0        0      0       
01200000 2009.09 17     19    22       24    0         0        0      0       
01200000 2009.10 17     19    22       24    0         0        0      0       
01200000 2009.11 17     19    22       24    0         0        0      0       
01200000 2009.12 17     19    22       24    0         0        0      0       
01200000 2010.01 17     19    22       24    0         0        0      0       
01200000 2010.02 17     19    22       24    0         0        0      0       
01200000 2010.03 17     19    22       24    0         0        0      0       
01200000 2010.04 17     19    22       24    0         0        0      0       
01200000 2010.05 17     19    22       2

In [22]:
allmerged:newjoinedtab ij `site_no`date xkey target
allmerged[`Action`Flood`Moderate`Major`elevation]:raze each allmerged[`Action`Flood`Moderate`Major`elevation]
allmerged[`site_no] :"F"$string each allmerged[`site_no]
allmerged[`lat`long]:raze each "F"$'string each allmerged[`lat`long]

In [23]:
pi:acos -1
allmerged:update month:`mm$date from allmerged
allmerged:update cos_t:cos 2*pi*date%12,
                 sin_t:sin 2*pi*date%12
                 from allmerged

In [24]:
/Join all dates within time range with stream sites so that there is no gaps in the time series
// x=table; y=col1; z=col2

ch_fn:{(count distinct x y)#'asc distinct x z}
all_dt_range:([]site_no:raze flip ch_fn[allmerged;`date;`site_no];
                date:raze ch_fn[allmerged;`site_no;`date])

show all_dt_merge:all_dt_range lj 2!allmerged

site_no date    ppt_absenergy ppt_abssumchange ppt_count ppt_countabovemean p..
-----------------------------------------------------------------------------..
1200000 2009.07 8.3659        11.36            31        10                 2..
1302020 2009.07 2.898         7.66             31        12                 1..
1303000 2009.07 2.8174        7.17             31        9                  2..
1303500 2009.07 3.2061        7.6              31        8                  2..
1304000 2009.07 7.5639        10.36            31        5                  2..
1304500 2009.07 6.7083        9.36             31        9                  2..
1305000 2009.07 9.9485        11.28            31        5                  2..
1305500 2009.07                                                              ..
1306460 2009.07 8.6007        11.2             31        5                  2..
1308000 2009.07 4.8711        9.27             31        6                  2..
1308500 2009.07 4.8711        9.27      

In [25]:
.ml.featurelag:{[x;c;l]
 if[(::)~c;c:.ml.i.fndcols[x;"f"]];
 v:raze{y xprev'x}[x c,:()]each l,:();
 max[l]_flip flip[x],(raze`$string[c],\:/:"_lag_",/:string l)!v}

In [26]:
ksll:key select by site_no,lat,long from all_dt_merge

In [27]:
lagfeats:{[x;y]t:select from y where site_no=x`site_no,lat=x`lat,long=x`long;
          2!.ml.featurelag[t;(::);1 2 3 12]}
allmerged:raze lagfeats[;all_dt_merge]each ksll

In [28]:
/
cols_fn   :{raze x[`cs]#'x y}
feat_fns  :{raze {x xprev raze y}[z]each ?[x;();();y]}
catch_tgts:0!select site_no,no_Flood,date,cs:count date by site_no from all_dt_merge

lagy  :feat_fns[catch_tgts;`no_Flood;12]
lag1  :feat_fns[catch_tgts;`no_Flood;1]
lag2  :feat_fns[catch_tgts;`no_Flood;2]
lag3  :feat_fns[catch_tgts;`no_Flood;3]
lagall:raze{count[x]mavg raze x}each?[catch_tgts;();();`no_Flood]
lagavg:avg each lag1,'lag2,'lag3

lagdt:([]site_no:cols_fn[catch_tgts;`site_no];date:cols_fn[catch_tgts;`date];
         lagged_target_1yr:ddlagy;lagged_target_recent:lag3;lagged_target_all:lagall)
show lagdt:delete from lagdt where i in where any flip null value each lagdt

catch_dts:0!select date,ppt_mean,ppt_max,ppt_var,cs:count i by site_no from all_dt_merge

avg1:feat_fns[catch_dts;`ppt_mean;1]
avg2:feat_fns[catch_dts;`ppt_mean;2]
avg3:feat_fns[catch_dts;`ppt_mean;3]

max1:feat_fns[catch_dts;`ppt_max;1]
max2:feat_fns[catch_dts;`ppt_max;2]
max3:feat_fns[catch_dts;`ppt_max;3]

var1:feat_fns[catch_dts;`ppt_var;1]
var2:feat_fns[catch_dts;`ppt_var;2]
var3:feat_fns[catch_dts;`ppt_var;3]

prevdt:([]site_no:raze cols_fn[catch_dts;`site_no];date:raze cols_fn[catch_dts;`date];
          avg_prev_1:avg1;avg_prev_2:avg2;avg_prev_3:avg3;
          max_prev_1:max1;max_prev_2:max2;max_prev_3:max3;
          var_prev_1:var1;var_prev_2:var2;var_prev_3:var3)

show prevdt:delete from prevdt where i in where any flip null value each prevdt

catch_atts:0!select site_no,ppt_mean,ppt_max,ppt_var,
             cs:count i by date,catch:2#'string each site_no 
             from all_dt_merge

avg1:feat_fns[catch_atts;`ppt_mean;1]
avg2:feat_fns[catch_atts;`ppt_mean;2]
avg3:feat_fns[catch_atts;`ppt_mean;3]

max1:feat_fns[catch_atts;`ppt_max;1]
max2:feat_fns[catch_atts;`ppt_max;2]
max3:feat_fns[catch_atts;`ppt_max;3]

var1:feat_fns[catch_atts;`ppt_var;1]
var2:feat_fns[catch_atts;`ppt_var;2]
var3:feat_fns[catch_atts;`ppt_var;3]

upstr:([]site_no:raze cols_fn[catch_dts;`site_no];date:raze cols_fn[catch_dts;`date];
         avg_ups_1:avg1;avg_ups_2:avg2;avg_ups_3:avg3;
         max_ups_1:max1;max_ups_2:max2;max_ups_3:max3;
         var_ups_1:var1;var_ups_2:var2;var_ups_3:var3)

show upstr:delete from upstr where i in where any flip null value each upstr

show allmerged:((allmerged ij 2!upstr) ij 2!prevdt) ij 2!lagdt

---

In [29]:
.aml.loadfile`:init.q

In [30]:
y:0<exec no_Flood from allmerged

In [31]:
alm:`site_no`date`lat`long xkey delete no_Action,no_Flood,no_Mod,no_Major from allmerged
tabreduced:2!key[alm],'.aml.preproc[value alm;y;`normal;`scale_fn`cols2scale!(`.ml.stdscaler;cols value alm)]

In [32]:
X:value tabreduced

In [33]:
mdls:.aml.models[`class;y]

In [34]:
d:`xv`prf`scf`k`seed!(`kfstrat;.ml.xv.fitpredict;`class`reg!(`.ml.accuracy;`.ml.mse);5;42)

In [35]:
r:.aml.runmodels[flip value flip 0^X;y;mdls;d]

Running model: AdaBoostClassifier


/home/deanna/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Running model: RandomForestClassifier
Running model: GradientBoostingClassifier
Running model: LogisticRegression
Running model: GaussianNB
Running model: KNeighborsClassifier
Running model: MLPClassifier
Running model: SVC
Running model: LinearSVC
Running model: BinaryKeras


Using TensorFlow backend.
2019-08-02 07:11:32.375290: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-08-02 07:11:32.376629: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


`AdaBoostClassifier`RandomForestClassifier`GradientBoostingClassifier`Logisti..


In [37]:
mm:mdls`model

In [41]:
`f1_score xdesc{(enlist[`mdl]!enlist y),@[;2]0!.ml.classreport . raze each flip x}'[r;mm]

mdl                        class     precision recall    f1_score  support
--------------------------------------------------------------------------
RandomForestClassifier     avg/total 0.9810524 0.938108  0.9584172 93541  
MLPClassifier              avg/total 0.9643532 0.9495858 0.9568304 93541  
GradientBoostingClassifier avg/total 0.9811285 0.9292758 0.9534924 93541  
SVC                        avg/total 0.9808049 0.919768  0.9478941 93541  
KNeighborsClassifier       avg/total 0.9645959 0.9171063 0.9393659 93541  
LogisticRegression         avg/total 0.9314827 0.8734978 0.9000562 93541  
BinaryKeras                avg/total 0.9083637 0.8788774 0.8929716 93541  
AdaBoostClassifier         avg/total 0.9340033 0.8598219 0.8928841 93541  
LinearSVC                  avg/total 0.8108545 0.8780894 0.8404459 93541  
GaussianNB                 avg/total 0.552502  0.7015733 0.3828532 93541  
